In [1]:
import time
from pathlib import Path
from datetime import datetime
import SimpleITK as sitk
from monai.utils import set_determinism
import math
import torch
from torch.utils.data import random_split, DataLoader
import monai
import gdown
import pandas as pd
import torchio as tio
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset,Dataset,PersistentDataset, list_data_collate, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
from pytorch_lightning.loggers import WandbLogger

sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()
from monai.transforms import (
    EnsureChannelFirstd,
    Orientationd,
    AsDiscrete,
    AddChanneld,
    Spacingd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureTyped,
    EnsureType,
    Resize,
    Resized,
    RandSpatialCropd
)

import time
from pathlib import Path
from datetime import datetime
import SimpleITK as sitk
from monai.utils import set_determinism
import math
import torch
from torch.utils.data import random_split, DataLoader
import monai
import gdown
import pandas as pd
import torchio as tio
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset,Dataset,PersistentDataset, list_data_collate, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import neptune.new as neptune

sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()
from monai.transforms import (
    EnsureChannelFirstd,
    Orientationd,
    AsDiscrete,
    AddChanneld,
    Spacingd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureTyped,
    EnsureType,
    Resize,
    Resized,
    RandSpatialCropd
)
from pytorch_lightning.loggers import NeptuneLogger


In [2]:
# run = neptune.init(
#     project="jakub.mitura14/picai",
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiN2YwYjFlZi1hNGM0LTQ4YjktYmU4NC0xMjNiMGQzZGJjOGYifQ==",
# ) 
# neptune_logger = NeptuneLogger(
#     project="picai",
#     prefix="baseline",  # custom prefix
# )
#run.stop()

In [3]:
# import wandb

# wandb.init(project="picai")

In [4]:
df = pd.read_csv('/home/sliceruser/labels/processedMetaData.csv')

df = df.loc[df['isAnyMissing'] ==False]
df = df.loc[df['isAnythingInAnnotated']>0 ]

def getMonaiSubjectDataFromDataFrame(row):
        """
        given row from data frame prepares Subject object from it
        """
        subject= {"adc": str(row['adc'])        
        , "cor":str(row['cor'])
        , "hbv":str(row['hbv'])
        , "sag":str(row['sag'])
        , "t2w":str(row['t2w'])
        , "isAnythingInAnnotated":row['isAnythingInAnnotated']
        , "patient_id":row['patient_id']
        , "study_id":row['study_id']
        , "patient_age":row['patient_age']
        , "psa":row['psa']
        , "psad":row['psad']
        , "prostate_volume":row['prostate_volume']
        , "histopath_type":row['histopath_type']
        , "lesion_GS":row['lesion_GS']
        , "label":str(row['reSampledPath'])
        
        
        }

        return subject


data_dicts = list(map(lambda row: getMonaiSubjectDataFromDataFrame(row[1])  , list(df.iterrows())))


In [5]:

train_transforms = Compose(
    [
        LoadImaged(keys=["t2w", "label"]),
        EnsureChannelFirstd(keys=["t2w", "label"]),
        Orientationd(keys=["t2w", "label"], axcodes="RAS"),
        Spacingd(keys=["t2w", "label"], pixdim=(
            1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
        tio.transforms.EnsureShapeMultiple((32 , 32, 32), include=["t2w", "label"]),
        #CropForegroundd(keys=["image", "label"], source_key="image"),
        RandCropByPosNegLabeld(
            keys=["t2w", "label"],
            label_key="label",
            spatial_size=(32, 32, 32),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="t2w",
            image_threshold=0,
        ),
        EnsureTyped(keys=["t2w", "label"]),
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["t2w", "label"]),
        EnsureChannelFirstd(keys=["t2w", "label"]),
        Orientationd(keys=["t2w", "label"], axcodes="RAS"),
        Spacingd(keys=["t2w", "label"], pixdim=(
            1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
        tio.transforms.EnsureShapeMultiple((32 , 32, 32), include=["t2w", "label"]),
        #CropForegroundd(keys=["image", "label"], source_key="image"),
        # RandCropByPosNegLabeld(
        #     keys=["t2w", "label"],
        #     label_key="label",
        #     spatial_size=(32, 32, 32),
        #     pos=1,
        #     neg=1,
        #     num_samples=4,
        #     image_key="t2w",
        #     image_threshold=0,
        # ),
        EnsureTyped(keys=["t2w", "label"]),
    ]
)

### Get Rid off cases where transforms lead to error for any reason

deficientPatIDs=[]
for dictt in data_dicts:    
    try:
        dat = train_transforms(dictt)
        dat = val_transforms(dictt)
        pass
    except:
        
        deficientPatIDs.append(dictt['patient_id'])
        print(dictt['patient_id'])

train_transforms(data_dicts[1])
def isInDeficienList(row):
        return row['patient_id'] not in deficientPatIDs

df["areTransformsNotDeficient"]= df.apply(lambda row : isInDeficienList(row), axis = 1)  

df = df.loc[ df['areTransformsNotDeficient']]

Possible incompatible factory load:
Running itk version :
itk version 5.2.1
Loaded factory version:
itk version 5.3.0
Loading factory:
/home/sliceruser/Slicer/bin/../lib/Slicer-5.0/ITKFactories/libMRMLIDIOPlugin.so




10294
11154
11447
11471


In [6]:



### Define Data Handling
class PiCaiDataModule(pl.LightningDataModule):
    def __init__(self,trainSizePercent,batch_size,num_workers,drop_last,df):
        super().__init__()
        self.batch_size = batch_size
        self.df = df
        self.num_workers = num_workers
        self.drop_last = drop_last
        self.train_set = None
        self.val_set = None
        self.test_set = None  
        self.trainSizePercent =trainSizePercent
        self.train_files = None
        self.val_files= None
        self.test_files= None
        self.train_ds = None
        self.val_ds= None
        self.test_ds= None        
        self.subjects= None        

    def splitDataSet(self,patList, trainSizePercent):
        """
        test train validation split
        TODO(balance sets)
        """
        totalLen=len(patList)
        train_test_split( patList  )
        numTrain= math.ceil(trainSizePercent*totalLen)
        numTestAndVal=totalLen-numTrain
        numTest=math.ceil(numTestAndVal*0.5)
        numVal= numTestAndVal-numTest

        # valid_set,test_set = torch.utils.data.random_split(test_and_val_set, [math. ceil(0.5), 0.5])
        print('Train data set:', numTrain)
        print('Test data set:',numTest)
        print('Valid data set:', numVal)
        return torch.utils.data.random_split(patList, [numTrain,numVal,numTest])

    def get_preprocessing_transform(self):
        preprocess = tio.Compose([
            tio.RescaleIntensity((-1, 1)),
            tio.CropOrPad(self.get_max_shape(self.subjects + self.test_subjects)),
            tio.EnsureShapeMultiple(8),  # for the U-Net
            tio.OneHot(),
        ])
        return preprocess
    
    def get_augmentation_transform(self):
        augment = tio.Compose([
            tio.RandomAffine(),
            tio.RandomGamma(p=0.5),
            tio.RandomNoise(p=0.5),
            tio.RandomMotion(p=0.1),
            tio.RandomBiasField(p=0.25),
        ])
        return augment

    def setup(self, stage=None):
        set_determinism(seed=0)
        self.subjects = list(map(lambda row: getMonaiSubjectDataFromDataFrame(row[1])   , list(df.iterrows())))
        train_set, valid_set,test_set = self.splitDataSet(self.subjects , self.trainSizePercent)
        self.train_subjects = train_set
        self.val_subjects = valid_set
        self.test_subjects = test_set
        self.train_ds =  Dataset(data=self.train_subjects, transform=train_transforms)
        self.val_ds=     Dataset(data=self.val_subjects, transform=val_transforms)
        self.test_ds=    Dataset(data=self.test_subjects, transform=val_transforms)    

    def train_dataloader(self):
        return DataLoader(self.train_ds, self.batch_size, drop_last=self.drop_last)

    def val_dataloader(self):
        return DataLoader(self.val_ds, self.batch_size, drop_last=self.drop_last)

    def test_dataloader(self):
        return DataLoader(self.test_ds, self.batch_size, drop_last=self.drop_last)#num_workers=self.num_workers,

data = PiCaiDataModule(
    df= df,
    batch_size=1,#TODO(batc size determined by lightning)
    trainSizePercent=0.7,
    num_workers=1,
    drop_last=True
)
data.prepare_data()
data.setup()

Train data set: 152
Test data set: 32
Valid data set: 32


In [7]:

class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class
    
    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer
    
    def prepare_batch(self, batch):
        return batch['t2w'], batch['label']
    
    def infer_batch(self, batch):
        x, y = self.prepare_batch(batch)
        y_hat = self.net(x)
        return y_hat, y

    def training_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        return loss


In [8]:
unet = monai.networks.nets.UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    #channels=(16, 16, 16, 16, 16),
    strides=(2, 2, 2, 2),
    num_res_units=1,
)

model = Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True),
    learning_rate=1e-2,
    optimizer_class=torch.optim.AdamW,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
)
trainer = pl.Trainer(
    #accelerator="cpu", #TODO(remove)
    max_epochs=5,
    #gpus=1,
    #precision=16, #TODO(unhash)
    callbacks=[early_stopping],
    #logger=wandb_logger,
    accelerator='gpu',
    devices=1
)
trainer.logger._default_hp_metric = False


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:

start = datetime.now()
print('Training started at', start)
trainer.fit(model=model, datamodule=data)
print('Training duration:', datetime.now() - start)

Training started at 2022-06-13 05:04:11.234422
Train data set: 152
Test data set: 32
Valid data set: 32


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | net       | UNet       | 2.4 M 
1 | criterion | DiceCELoss | 0     
-----------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.792     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
single channel prediction, `softmax=True` ignored.
Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


Training: 0it [00:00, ?it/s]

TypeError: list indices must be integers or slices, not str

In [ ]:
cache_dir='/home/sliceruser/preprocess'
class PiCaiDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, df,trainSizePercent):
        super().__init__()
        self.batch_size = batch_size
        self.df = df
        self.trainSizePercent =trainSizePercent
        self.subjects = None
        self.train_subjects = None
        self.val_subjects = None
        self.test_subjects = None
        self.preprocess = None
        self.transform = None
        
        self.train_set = None
        self.val_set = None
        self.test_set = None        

        #adding temporarly simplified dictionaries
        train_files = None
        val_files= None
        test_files= None
        train_ds = None
        val_ds= None
        test_ds= None


    def splitDataSet(self,patList, trainSizePercent):
        """
        test train validation split
        TODO(balance sets)
        """
        totalLen=len(patList)
        train_test_split( patList  )
        numTrain= math.ceil(trainSizePercent*totalLen)
        numTestAndVal=totalLen-numTrain
        numTest=math.ceil(numTestAndVal*0.5)
        numVal= numTestAndVal-numTest

        # valid_set,test_set = torch.utils.data.random_split(test_and_val_set, [math. ceil(0.5), 0.5])
        print('Train data set:', numTrain)
        print('Test data set:',numTest)
        print('Valid data set:', numVal)
        return torch.utils.data.random_split(patList, [numTrain,numVal,numTest])
        
    def get_max_shape(self, subjects):
        dataset = tio.SubjectsDataset(subjects)
        shapes = np.array([s.spatial_shape for s in dataset])
        return shapes.max(axis=0)
    
    def prepare_data(self):
        set_determinism(seed=0)
        self.subjects = list(map(lambda row: getMonaiSubjectDataFromDataFrame(row[1])   , list(df.iterrows())))
        train_set, valid_set,test_set = self.splitDataSet(self.subjects , self.trainSizePercent)
        self.train_subjects = train_set
        self.val_subjects = valid_set
        self.test_subjects = test_set
        #adding temporarly simplified dictionaries
        self.train_files = [
            {"image": pat['t2w']  , "label": pat['label']  }
            for pat  in self.train_subjects
        ]
        self.val_files= [{"image": pat['t2w']  , "label": pat['label']  }
            for pat  in self.val_subjects
        ]
        self.test_files= [{"image": pat['t2w']  , "label": pat['label']  }
            for pat  in self.test_subjects
        ]

    
    def get_preprocessing_transform(self):
        """
        preprocessing based on https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7790158/ https://github.com/NIH-MIP/Radiology_Image_Preprocessing_for_Deep_Learning/blob/main/Codes/Main_Preprocessing.py
        denoising - https://github.com/ketanfatania/QMRI-PnP-Recon-POC     ;   https://github.com/AurelienCD/Resampling_Denoising_Deep_Learning_MRI   https://github.com/AurelienCD/Resampling_Denoising_Deep_Learning_MRI
        bias field correction -https://discourse.itk.org/t/n4-bias-field-correction/3972
        registration and superresolotion - https://github.com/gift-surg/NiftyMIC
        rest of registration- https://github.com/SuperElastix/elastix
        standardization - https://github.com/NIH-MIP/Radiology_Image_Preprocessing_for_Deep_Learning/blob/main/Codes/Main_Preprocessing.py
        My:
        resampling -  https://github.com/AurelienCD/Resampling_Denoising_Deep_Learning_MRI
        """
        torchio_transforms = tio.transforms.EnsureShapeMultiple(8, include=["image"]) # for unet
        val_transforms = Compose(
            [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        EnsureTyped(keys=["image", "label"],data_type='tensor', dtype=torch.float),
        torchio_transforms,
        Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),#TODO(make more refined)
        CropForegroundd(keys=["image", "label"], source_key="image"),
        
            ]
                )
        #TODO add         
        # preprocess = tio.Compose([
        #     tio.RescaleIntensity((-1, 1)),
        #     tio.CropOrPad(self.get_max_shape(self.subjects)),
        #     tio.EnsureShapeMultiple(8),  # for the U-Net
        #     tio.OneHot(),
        # ])


        return val_transforms
    
    # def get_augmentation_transform(self):
    #     #TODO(use augmentations)
    #     # augment = tio.Compose([
    #     #     tio.RandomAffine(),
    #     #     # tio.RandomGamma(p=0.5),
    #     #     # tio.RandomNoise(p=0.5),
    #     #     tio.RandomMotion(p=0.1),
    #     #     tio.RandomBiasField(p=0.25),
    #     # ])
    #     return augment

    def setup(self, stage=None):
        self.preprocess = self.get_preprocessing_transform()
        # augment = self.get_augmentation_transform()
        #TODO(add augmentations) 
        self.train_ds =  PersistentDataset(
            data=self.train_files, transform=self.preprocess,cache_dir=cache_dir   )
        self.val_ds=  PersistentDataset(
            data=self.train_files, transform=self.preprocess,cache_dir=cache_dir)
        self.test_ds=  PersistentDataset(
             data=self.train_files, transform=self.preprocess,cache_dir=cache_dir)

        # self.train_set = tio.SubjectsDataset(self.train_subjects, transform=self.transform)
        # self.val_set = tio.SubjectsDataset(self.val_subjects, transform=self.preprocess)
        # self.test_set = tio.SubjectsDataset(self.test_subjects, transform=self.preprocess)

    def train_dataloader(self):
        return DataLoader(self.train_ds, self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_ds, self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_ds, self.batch_size)

In [ ]:
data = PiCaiDataModule(
    df= df,
    batch_size=2,#TODO(batc size determined by lightning)
    trainSizePercent=0.7
)
data.prepare_data()
data.setup()
# print('Training:  ', len(data.train_set))
# print('Validation: ', len(data.val_set))
# print('Test:      ', len(data.test_set))

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class
    
    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer
    
    def prepare_batch(self, batch):
        return batch['image'], batch['label']
    
    def infer_batch(self, batch):
        x, y = self.prepare_batch(batch)
        y_hat = self.net(x)
        return y_hat, y

    def training_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        return loss

In [ ]:
unet = monai.networks.nets.UNet(
    dimensions=3,
    in_channels=1,
    out_channels=2,
    channels=(8, 16, 32, 64),
    strides=(2, 2, 2),
)

model = Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True),
    learning_rate=1e-2,
    optimizer_class=torch.optim.AdamW,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
)
trainer = pl.Trainer(
    accelerator="cpu", #TODO(remove)
    max_epochs=5,
    gpus=1,
    #precision=16, #TODO(unhash)
    callbacks=[early_stopping],
)
trainer.logger._default_hp_metric = False

In [ ]:
# start = datetime.now()
# print('Training started at', start)
# trainer.fit(model=model, datamodule=data)
# print('Training duration:', datetime.now() - start)

In [ ]:
import torch
a = torch.LongTensor(1).random_(0, 10)
a = a.to(device='cuda')
torch.cuda.is_available()

In [ ]:
def getSpacing(row,strr):
    reSampledPath=str(row[1][strr])
    image = sitk.ReadImage(reSampledPath)
    print( strr)
    print(reSampledPath)
    print(image.GetSpacing())

def GetSizee(row,strr):
    reSampledPath=str(row[1][strr])
    image = sitk.ReadImage(reSampledPath)
    print( strr)
   # print(reSampledPath)
    print(image.GetSize())

def GetSizecomp(row):
    reSampledPath=str(row[1]['reSampledPath'])
    image = sitk.ReadImage(reSampledPath)
    sizzA=image.GetSize()

    reSampledPath=str(row[1]['t2w'])
    image = sitk.ReadImage(reSampledPath)
    sizzB=image.GetSize()

    return sizzA==sizzB


row = list(df.iterrows())[10]
strr='reSampledPath'
# reSampledPath=row[strr]
# reSampledPath
# GetSizee(row,'reSampledPath')
# GetSizee(row,'t2w')

#GetSizecomp(row,'reSampledPath', 't2w')


def GetSizecompMap(row):
    reSampledPath=str(row['reSampledPath'])
    t2wPath=str(row['t2w'])

    if(len(reSampledPath)>1):
        image = sitk.ReadImage(reSampledPath)
        sizzA=image.GetSize()
        image = sitk.ReadImage(t2wPath)
        sizzB=image.GetSize()
        return sizzA==sizzB
    return True

# df["isT2wAndLabelSame"]= df.apply(lambda row : GetSizecompMap(row), axis = 1)  
# df["isT2wAndLabelSame"]

patList = list(map(lambda row: getMonaiSubjectDataFromDataFrame(row[1])   , list(df.iterrows())))
patList[0]

#data_dict = LoadImaged(keys="img")({"img":row[1]['t2w']})
row[1]['t2w']
#LoadImaged(keys=["t2w", "reSampledPath"])(patList[20]  )
# data_dict = LoadImaged(keys="img")({"img":'/home/sliceruser/data/10011/10011_1000011_t2w.mha'})
# data_dict
sitk.ReadImage('/home/sliceruser/data/10011/10011_1000011_t2w.mha')